# Chapter 13 듀얼리티 활용

## 13.1 들어가며

듀얼리티는 두 도메인간의 관계를 뜻한다. 기계번역은 도메인간의 정보량의 차이가 거의 없는 편이다.

### 13.1.1 CycleGAN

X 도메인의 이미지를 전체적인 틀을 크게 바꾸지 않는 선에서 Y 도메인의 이미지로 변환하는 방법이다. 각 도메인 별로 생성자와 판별자가 존재해서 각각 변환한 결과를 실제 이미지와 비교하면서 훈련한다.

## 13.2 듀얼리티를 활용한 지도학습(DSL)

teacher forcing의 문제로 생기는 어려움을 듀얼리티에서 regularization term을 이끌어내 해결한다. 베이즈 정리에 따라 다음 수식을 만족해야 한다.

$$P(x)P(y|x;\theta_{x \to y}) = P(y)P(x|y;\theta_{y \to x})$$

이를 변역 훈련의 목표에 적용하면 다음과 같이 된다.

$$objective1 : min_{\theta_{x \to y}} \frac 1 n \sum_{i=1}^n l_1(f(x_i;\theta_{x \to y}), y_i)$$
$$objective2 : min_{\theta_{y \to x}} \frac 1 n \sum_{i=1}^n l_2(g(y_i;\theta_{y \to x}), x_i)$$

$$L_{duality} = |(log \hat P(x) + logP(y|x;\theta_{x \to y})) - (log \hat P(y) + log P(x|y;\theta_{y \to x}))|^2$$

$logP(y|x;\theta_{x \to y}))$와 $log P(x|y;\theta_{y \to x}))$는 동시에 훈련시키는 신경망 가중치 파라미터를 통해 구하고 $log \hat P(x)$와 $log \hat P(y)$는 단일 언어 코퍼스를 통해 별도로 훈련된 모델을 통해서 근사할 수 있다.

$$\theta_{x \to y} \leftarrow \theta_{x \to y} - \gamma \nabla_{\theta_{x \to y}} \frac 1 n \sum_{i=1}^n [l_1(f(x_i;\theta_{x \to y}), y_i) + \lambda_{x \to y}L_{duality}]$$
$$\theta_{y \to x} \leftarrow \theta_{y \to x} - \gamma \nabla_{\theta_{y \to x}} \frac 1 n \sum_{i=1}^n [l_2(g(y_i;\theta_{y \to x}), x_i) + \lambda_{y \to x}L_{duality}]$$

$\lambda$로 손실함수 내의 비율을 조절할 수 있다. $\lambda$가 크면 regularization term을 최소화하는데 집중하게 될 것이고 0.01일 때 대체적으로 가장 좋은 성능을 보였다.

DSL이 NMT와 MRT보다 높은 성능을 보였다.

### 13.2.1 파이토치 예제 코드

#### simple_nmt와 dual_trainer.py

MRT와 동일하게 seq2seq 모델구조는 변하지 않고 훈련방식만 달라진다.

In [7]:
import numpy as np
import torch

from torch import optim
import torch.nn.utils as torch_utils
# from ignite.engine import Engine, Events

VERBOSE_SILENT = 0
VERBOSE_EPOCH_WISE = 1
VERBOSE_BATCH_WISE = 2

In [1]:
# 동시에 학습하는 것을 하드 코딩하는 것을 막기 위해서 변수 선언
X2Y, Y2X = 0, 1

In [3]:
def _reordering(self, x, y, l):
    # 인코더는 packe_sequence를 처리할 때 길이로 정렬해서 처리한다.
    # 반대 방향 모델을 학습할 때 다시 정렬이 필요하기 때문에 재정렬하고 원래 순서로 복원할 수 있도록 한다.
    
    # 길이로 정렬
    indice = l.topk(l.size(0))[1]
    
    # 재정렬
    x_ = x.index_select(dim=0, index=indice).contiguous()
    y_ = y.index_select(dim=0, index=indice).contiguous()
    l_ = l.index_select(dim=0, index=indice).contiguous()
    
    # 재정렬한 것을 복원할 정보 생성
    restore_indice = (-indice).topk(l.size(0))[1]
    
    return x_, y_, l_, restore_indice

In [4]:
def _get_loss(self, x, y, x_hat, y_hat, x_lm=None, y_lm=None, lagrange=1e-3):
    # |x| = (batch_size, length0)
    # |y| = (batch_size, length1)
    # |x_hat| = (batch_size, length0, output_size0)
    # |y_hat| = (batch_size, length1, ouput_size1)
    # |x_lm| = |x_hat|
    # |y_lm| = |y_hat|
    
    losses = []
    losses += [self.crits[X2Y](y_hat.contiguous().view(-1, y_hat.size(-1)),
                               y.contiguous().view(-1)
                               )]
    losses += [self.crits[Y2X](x_hat.contiguous().view(-1, x_hat.size(-1)),
                               x.contiguous().view(-1)
                               )]
    
    # |losses[X2Y]| = (batch_size * length1)
    # |losses[Y2X]| = (batch_size * lenght0)
    
    losses[X2Y] = losses[X2Y].view(y.size(0), -1).sum(dim=-1)
    losses[Y2X] = losses[Y2X].view(x.size(0), -1).sum(dim=-1)
    # |losses[X2Y]| = (batch_size)
    # |losses[Y2X]| = (batch_size)
    
    if x_lm is not None and y_lm is not None:
        lm_losses = []
        lm_losses += [self.crits[X2Y](y_lm.contiguous().view(-1, y_lm.size(-1)),
                                      y.contiguous().view(-1)
                                      )]
        lm_losses += [self.cirts[Y2X](x_lm.contiguous().view(-1, x_lm.size(-1)),
                                      x.contiguous().view(-1)
                                      )]
        # |lm_losses[X2Y]| = (batch_size * length1)
        # |lm_losses[Y2X]| = (batch_size * length0)
        
        lm_losses[X2Y] = lm_losses[X2Y].view(y.size(0), -1).sum(dim=-1)
        lm_losses[Y2X] = lm_losses[Y2X].view(x.size(0), -1).sum(dim=-1)
        # |lm_losses[X2Y]| = (batch_size)
        # |lm_losses[Y2X]| = (batch_size)
        
        # just for information
        dual_loss = lagrange * ((-lm_losses[Y2X] + -losses[X2Y].detach()) - \
                                (-lm_losses[X2Y] + -losses[Y2X].detach()))**2
        
        dual_loss_x2y = lagrange * ((-lm_losses[Y2X] + -losses[X2Y]) - \
                                    (-lm_losses[X2Y] + -losses[Y2X].detach()))**2
        dual_loss_y2x = lagrange * ((-lm_losses[Y2X] + -losses[X2Y].detach()) - \
                                    (-lm_losses[X2Y] + -losses[Y2X]))**2
        
        losses[X2Y] += dual_loss_x2y
        losses[Y2X] += dual_loss_y2x
        
    if x_lm is not None and y_lm is not None:
        return losses[X2Y].sum(), losses[Y2X].sum(), dual_loss.sum()
    else:
        return losses[X2Y].sum(), losses[Y2X].sum(), None

In [8]:
def train_epoch(self, train, optimizers, no_regularization=True, verbose=VERBOSE_BATCH_WISE):
    '''1 epoch에 대해 훈련 수행하는 함수
    '''
    total_loss, total_word_count = 0, 0
    total_grad_norm = 0
    avg_loss, avg_grad_norm = 0, 0
    sample_cnt = 0
    
    progress_bar = tqdm(train, desc='Training: ', unit='batch') if verbos is VERBOSE_BATCH_WISE else train
    # 전체 데이터셋에 대해 반복
    for idx, mini_batch in enumerate(progress_bar):
        # raw target variable은 BOS와 EOS가 있다.
        # seq2seq의 output에는 BOS가 없어야 한다.
        # 따라서 추론 때는 BOS를 없애준다.
        
        # 다음 스텝의 gradient descent 전에model parameters의 그래디언트를 초기화한다.
        optimizers[X2Y].zero_grad()
        optimizers[Y2X].zero_grad()
        
        x_0, y_0 = (mini_batch.src[0][:, 1:-1],
                    mini_batch.src[1] - 2
                    ), mini_batch.tgt[0][:, :-1]
        # |x_0| = (batch_size, length0)
        # |y_0| = (batch_size, length1)
        y_hat = self.models[X2Y](x_0, y_0)
        # |y_hat| = (batch_size, length1, output_size1)
        with torch.no_grad():
            y_lm = self.language_models[X2Y](y_0)
            # |y_lm| = |y_hat|
            
        # src와 tgt 반전
        x_0, y_0_0, y_0_1, restore_indice = self._reordering(mini_batch.src[0][:, :-1],
                                                             mini_batch_tgt[0][:, 1:-1],
                                                             mini_batch_tgt[1] - 2)
        y_0 = (y_0_0, y_0_1)
        # |x_0| = (batch_size, length0)
        # |y_0| = (batch_size, length1)
        x_hat = self.models[Y2X](y_0, x_0).index_select(dim=0, index=restore_indice)
        # |x_hat| = (batch_size, length0, ouput_size0)
        
        with torch.no_grad():
            x_lm = self.language_models[Y2X](x_0)
            # |x_lm| = |x_hat|
            
        x, y = mini_batch.src[0][:, 1:], mini_batch.tgt[0][:, 1:]
        # DSL은 warm-started로 학습해야되기 때문에 시작할 때 regularization을 꺼준다.
        losses  = self._get_loss(x, y, x_hat, y_hat, x_lm, y_lm, lagrange=self.config.dsl_lambda if not no_regularization else .0)
        
        losses[X2Y].div(y.size(0)).backward()
        losses[Y2X].div(x.size(0)).backward()
        
        word_count = int((mini_batch.src[1].detach().sum()) + 
                         (mini_batch.tgt[1].detach().sum())
                         )
        loss = float(losses[X2Y].detach() + losses[Y2X].detach()) - float(losses[-1].detach() * 2)
        param_norm = float(utils.get_parameter_norm(self.models[X2Y].parameters()).detach() + 
                           utils.get_parameter_norm(self.models[Y2X].parameters()).detach())
        grad_norm = float(utils.get_grad_norm(self.models[X2Y].parameters()).detach() +
                          utils.get_grad_norm(self.models[Y2X].parameters()).detach())
        
        total_loss += loss
        total_word_count += word_count
        total_grad_norm += grad_norm
        
        avg_loss = total_loss / total_word_count
        avg_grad_norm = total_grad_norm / (idx + 1)
        
        if verbose is VERBOSE_BATCH_WISE:
            progress_bar.set_postfix_str('|param|=%.f |g_param|=%.2f loss=%.4e PPL=%.2f' % (param_norm,
                                                                                            grad_norm,
                                                                                            loss / word_count,
                                                                                            exp(avg_loss)
                                                                                           ))
        
        # 그래디언트 폭발 문제를 막기 위해서 그래디언트 클리핑을 해준다.
        torch_utils.clip_grad_norm_(self.models[X2Y].parameters(),
                                    self.config.max_grad_norm
                                    )
        torch_utils.clip_grad_norm_(self.models[Y2X].parameters(),
                                    self.config.max_grad_norm
                                    )
        
        optimizers[X2Y].step()
        optimizers[Y2X].step()
        
        sample_cnt += mini_batch.tgt[0].size(0)
        
        if idx >= len(progress_bar) * self.config.train_ratio_per_epoch:
            break
            
    if verbose is VERBOSE_BATCH_WISE:
        progress_bar.close()
    
    return avg_loss, param_norm, avg_grad_norm

### 13.2.2 실험 결과

teacher forcing을 사용하면서 가단한 regularization term 손실함수에 더하면서 성능이 개선된 기계번역 모델을 학습할 수 있다.

## 13.3 듀얼리티를 활용한 비지도학습

### 13.3.1 듀얼 러닝 기계번역

**Dual Learning for Machine Translation** 논문
- CycleGAN과 비슷
- 단일 언어 코퍼스로부터 받은 문장을 번역하고, 번역된 문장을 반대방향 번역으로 복원했을 때, 복원된 문장이 원래 처음 문장과의 차이가 최소화되도록 훈련

$$\theta_{AB} \leftarrow \theta_{AB} + \gamma \nabla_{\theta_{AB}} \hat {\mathbb E} [r]$$
$$\theta_{BA} \leftarrow \theta_{BA} + \gamma \nabla_{\theta_{BA}} \hat {\mathbb E} [r]$$

- 병렬 코퍼스로 사전훈련된 생성자는 폴리시 그래디언트를 활용하여 업데이트 가능

$$r = \gamma \cdot r_{AB} + (1-\gamma) \cdot r_{BA}$$
$$where \ r_{AB} = LM_B(s_{mid})$$
$$and \ r_{BA} = logP(s|s_{mid} ; \theta_{BA})$$
- k개의 샘플링한 문장에 대해 각 방향에 대한 보상을 구한 후 선형결합
- 보상에 대한 기대값을  각 파라미터에 대해 미분한 후 업데이트 수식에 대입
- 반대방향도 비슷하게 수행
- NMT, NMT+Back Translation와 비교 했을 때 듀얼 러닝이 가장 높았다. 다만 병렬 코퍼스 양이 많아지면 성능 폭 줄어듦

### 13.3.2 듀얼 비지도학습

주변 분포의 성질을 이용해서 제약 조건을 만든다.

$$P(y) = \sum_{x \in X} P(x, y) = \sum_{x \in X} P(y|x) P(x)$$

몬테카를로 샘플링으로 근사할 수 있다.

$$\begin{align}
P(y) & = \sum_{x \in X} P(y|x;\theta)P(x)= \mathbb E_{x \sim P(x)}[P(y|x;\theta)] \\
& \approx \frac 1 K \sum_{i=1}^{K} P(y|x^i;\theta) \\
\end{align}$$

$$Objective : \sum_{n=1}^N logP(y^n|x^n;\theta),$$
$$s.t. P(y) = \mathbb E_{x \sim P(x)} [P(y|x;\theta)], \forall y \in \mathcal M$$

$$S(\theta) = [log \hat P(y) - log \mathbb E_{x~\hat P(x)}[P(y|x;\theta)]]^2$$
$$L(\theta) = - \sum_{n=1}^N log P(y^n|x^n;\theta) + \lambda S(\theta)$$

중요도표집을 통해서 타깃 언어의 문장을 반대 방향 번역기에 넣어 샘플링해서 P(y)를 구한다.

$$\begin{align}
P(y) & = \mathbb E_{x \sim \hat P(x)}[P(y|x;\theta)] = \sum_{x \in X} P(y|x;\theta) \hat P(x) \\
& = \sum_{x \in X} \frac {P(y|x;\theta)\hat P(x)} {P(x|y)} P(x|y) \\
& = \mathbb E_{x \sim P(x|y)} [\frac {P(y|x;\theta) \hat P(x)} {P(x|y)}] \\
& \approx \frac 1 K \sum_{i=1}^K \frac {P(y|x;\theta) \hat P(x_i)} {P(x_i|y)}, x_i \sim P(x|y) \\
\end{align}$$

전체 손실 함수
$$L(\theta) \approx - \sum_{n=1}^N logP(y^n|x^n;\theta) + \lambda \sum_{s=1}^S [log \hat P(y^s) - log \frac 1 K \sum_{i=1}^K \frac {\hat P(x_i) P(y^s|x_i \theta)} {P(x_i|y^s)}]^2$$

DUL이 기존 방법들보다 성능을 모두 앞질렀다.

## 13.4 back-translation 재해석하기

back-translation을 듀얼리티 관점에서 해석

$$L(\theta) = - \sum_{n=1}^N logP(y^n|x^n;\theta) - \sum_{s=1}^S log P(y^s)$$

젠센 부등식에 따라 다음 성립
$$\begin{align}
log P(y) & = log \sum_{x \in X} P(y|x) P(x) \\
& = log \sum_{x \in X} P(x|y) \frac {P(y|x)P(x)} {P(x|y)} \\
& \ge \sum_{x \in X} P(x|y) log \frac {P(y|x)P(x)} {P(x|y)} \\
& = \mathbb E_{x \sim P(x|y)} [log P(y|x) + log \frac {P(x)} {P(x|y)}] \\
& = \mathbb E_{x \sim P(x|y)} [log P(y|x)] + \mathbb E_{x \sim P(x|y)} [log \frac {P(x)} {P(x|y)}] \\
& = \mathbb E_{x \sim P(x|y)} [log P(y|x)] - KL(P(x|y)||P(x)) \\
\end{align}$$

비용 함수 다음 성립
$$\begin{align}
\mathcal L(\theta) & \le -\sum_{n=1}^N log P(y^n|x^n;\theta) - \sum_{s=1}^S (\mathbb E_{x \sim P(x|y^S)} [log P(y^S|x;\theta)] - KL(P(x|y^S) || P(x))) \\
& \approx -\sum_{n=1}^N log P(y^n|x^n;\theta) -\frac 1 K \sum_{s=1}^S \sum_{i=1}^K log P(y^S | x_i;\theta) + \sum_{s=1}^S KL(P(x|y^S) || P(x)) \\
& = \tilde {\mathcal L} (\theta) \\
\end{align}$$

$\mathcal {\tilde L}(\theta)$를 최소화하는 것은 $\mathcal L(\theta)$를 최소화하는 것과 같은 효과

새로운 손실 함수 미분, KLD 부분은 상수이므로 생략됨

$$\nabla_\theta \tilde {\mathcal L}(\theta) = - \sum_{n=1}^N \nabla_\theta log P(y^n|x^n;\theta) - \frac 1 K \sum_{s=1}^S \sum_{i=1}^K \nabla_\theta log P(y^S|x_i;\theta)$$

첫번째 항은 $x^n$이 주어졌을 때, $y^n$의 확률을 최대로 하는 $\theta$를 찾는 것. 두번재항은 샘플링된 $x^i$가 주어졌을 때 단일 언어 포퍼스 문장 $y^S$가 나올 평균 확률을 최대로 하는 $\theta$를 찾는 것.